## Old (Loader)

In [1]:
import os
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
from constants import EVAL_ANSWERS, EVAL_QUESTIONS
from ragas.metrics.critique import harmfulness

In [3]:
from langchain_core.chat_history import BaseChatMessageHistory
from conversation import CustomChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = CustomChatMessageHistory()
    return store[session_id]

In [4]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from rag import caller_with_context, chain_with_context

full_chain_with_context = chain_with_context()
full_chain_with_context_and_message_history = RunnableWithMessageHistory(
    full_chain_with_context,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

In [5]:
sess_id = "X123"

result = full_chain_with_context_and_message_history.invoke(
        {"question": "Apa saja yang termasuk dalam Bank Pelapor?"},
        config={"configurable": {"session_id": sess_id}})

{'result': 'ketentuan_terkait', 'question': 'Apa saja yang termasuk dalam Bank Pelapor?', 'history': []}


c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Error in RootListenersTracer.on_chain_end callback: KeyError('output')


In [6]:
result

{'query': 'Apa saja yang termasuk dalam Bank Pelapor?',
 'result': 'Dalam konteks yang diberikan, tidak ada informasi yang menjelaskan mengenai apa yang termasuk dalam Bank Pelapor.',
 'source_documents': [Document(page_content='Bank yang bersangkutan”, dengan mencantumkan keterangan bahwa\npencairannya hanya dapat dilakukan setelah mendapat persetujuan\ntertulis dari Dewan Gubernur Bank Indonesia; dan\nj. surat pernyataan dari calon pemegang saham bagi Bank yang berbentuk badan hukum Perseroan Terbatas/Perusahaan Daerah atau dari calon anggota bagi Bank yang berbentuk badan hukum Koperasi, bahwa setoran modal sebagaimana dimaksud dalam huruf i:\n1. tidak berasal dari pinjaman atau fasilitas pembiayaan dalam bentuk\napapun dari Bank dan/atau pihak lain di Indonesia; dan/atau\n2. tidak berasal dari dan untuk tujuan pencucian uang (money laundering).\n(2) Daftar calon pemegang saham atau daftar calon anggota sebagaimana dimaksud pada ayat (1) huruf b:\na. dalam hal perorangan wajib diser

In [7]:
sess_id = "X123"

result = full_chain_with_context_and_message_history.invoke(
        {"question": "Apa saja yang termasuk dalam Bank Pelapor?"},
        config={"configurable": {"session_id": sess_id}})

{'result': 'ketentuan_terkait', 'question': 'Apa saja yang termasuk dalam Bank Pelapor?', 'history': []}


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy, context_recall
from ragas.langchain import RagasEvaluatorChain

# make eval chains
eval_chains = {
    m.name: RagasEvaluatorChain(metric=m) 
    for m in [faithfulness, answer_relevancy, context_relevancy, context_recall]
}


## Evaluation Loading

In [8]:
queries = EVAL_QUESTIONS
ground_truths = EVAL_ANSWERS

In [ ]:
sess_id = "X123"
results = []
contexts = []

for query in queries:
    result = full_chain_with_context_and_message_history.invoke(
        {"question": query},
        config={"configurable": {"session_id": sess_id}})
    
    print(result)
    results.append(result['answer'])
    sources = result["context"]
    contents = []
    for i in range(len(sources)):
        contents.append(sources[i][0].page_content)
    contexts.append(contents)

d = {
    "question": queries,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)

In [ ]:
import json

with open("dict_eval.json", "w") as outfile: 
    json.dump(d, outfile)


## Evaluation (Start)

In [1]:
import nest_asyncio
nest_asyncio.apply()


In [2]:
from azure_config import azure_embeddings, azure_llm

llm = azure_llm()
embeddings = azure_embeddings()

In [3]:
import json
with open('dict_eval.json') as json_file:
    d = json.load(json_file)


In [4]:
from datasets import Dataset
dataset = Dataset.from_dict(d)

In [5]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 15
})

In [6]:
df = dataset.to_pandas()
df.head()

,question,answer,contexts,ground_truth
0,Apa batas waktu yang diberikan untuk persetuju...,Batas waktu yang diberikan untuk persetujuan a...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Batas waktu yang diberikan adalah paling lamba...
1,Berapa lama waktu yang diberikan bagi bank yan...,Bank yang telah mendapat izin usaha dari Guber...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Bank wajib melakukan kegiatan usaha perbankan ...
2,Siapa yang dapat mendirikan dan memiliki Bank ...,Bank Perantara hanya dapat didirikan dan dimil...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Bank Perantara hanya dapat didirikan dan dimil...
3,Apa yang terjadi jika dokumen permohonan izin ...,Jika dokumen permohonan izin usaha tidak lengk...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Persetujuan atau penolakan tidak akan diproses...
4,Apa saja yang termasuk dalam Bank Pelapor?,"Dalam konteks yang diberikan, tidak ada inform...","[Bank yang bersangkutan”, dengan mencantumkan ...",Bank Pelapor terdiri dari: 1.) Kantor pusat Ba...


In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_similarity, 
    answer_correctness
)

from ragas import evaluate

result = evaluate(
    amnesty_qa["eval"],
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_similarity, 
        answer_correctness
    ],
)

result

In [7]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate

score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm, embeddings=embeddings,    )

Evaluating:   0%|          | 0/105 [00:00<?, ?it/s]

In [9]:
df = score.to_pandas()
df


,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,answer_similarity,answer_correctness,harmfulness
0,Apa batas waktu yang diberikan untuk persetuju...,Batas waktu yang diberikan untuk persetujuan a...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Batas waktu yang diberikan adalah paling lamba...,1.000000,0.990749,1.000000,1.0,0.951712,0.737928,0
1,Berapa lama waktu yang diberikan bagi bank yan...,Bank yang telah mendapat izin usaha dari Guber...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Bank wajib melakukan kegiatan usaha perbankan ...,1.000000,0.993915,1.000000,1.0,0.943906,0.610977,0
2,Siapa yang dapat mendirikan dan memiliki Bank ...,Bank Perantara hanya dapat didirikan dan dimil...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Bank Perantara hanya dapat didirikan dan dimil...,1.000000,0.999999,0.976190,1.0,0.994458,0.998615,0
3,Apa yang terjadi jika dokumen permohonan izin ...,Jika dokumen permohonan izin usaha tidak lengk...,[Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pa...,Persetujuan atau penolakan tidak akan diproses...,1.000000,0.000000,0.916667,1.0,0.891383,0.722846,0
4,Apa saja yang termasuk dalam Bank Pelapor?,"Dalam konteks yang diberikan, tidak ada inform...","[Bank yang bersangkutan”, dengan mencantumkan ...",Bank Pelapor terdiri dari: 1.) Kantor pusat Ba...,0.000000,0.000000,1.000000,0.0,0.839897,0.209974,0
5,Apa saja jenis data yang wajib dilaporkan dala...,Jenis data yang wajib dilaporkan dalam Laporan...,[a. Bank Indonesia c.q. Unit Khusus Manajemen ...,Jenis data yang wajib dilaporkan dalam LHBU me...,1.000000,0.993323,0.525000,1.0,0.939475,0.609869,0
6,Apa yang harus dilakukan jika penyampaian dan/...,Jika penyampaian dan/atau koreksi LHBU dilakuk...,[Nomor Ketentuan: 13/3/DPM/2011\nKetentuan: Ro...,Jika penyampaian dan/atau koreksi LHBU dilakuk...,1.000000,0.899678,1.000000,1.0,0.945390,0.486353,0
7,Apa itu Data JIBOR dan bagaimana penetapannya ...,Data JIBOR adalah data suku bunga penawaran pa...,[Nomor Ketentuan: 13/3/DPM/2011\nKetentuan: Ro...,Data Jakarta Interbank Offered Rate (JIBOR) di...,1.000000,0.937445,0.876190,1.0,0.953683,0.738421,0
8,Berapa persentase minimal Dana Pendidikan dari...,Dana Pendidikan minimal dari Anggaran Pengelua...,[Nomor Ketentuan: 31/310/KEP/DIR/1999\nKetentu...,Sekurang-kurangnya sebesar 5'%' dari Anggaran ...,1.000000,0.997516,0.916667,1.0,0.914711,0.728678,0
9,Apa yang harus dilakukan Bank jika terdapat si...,Bank harus menyetorkan sisa Dana Pendidikan te...,[Nomor Ketentuan: 31/310/KEP/DIR/1999\nKetentu...,Bank wajib menyetorkan Dana Pendidikan tersebu...,0.333333,0.991774,1.000000,1.0,0.983532,0.620883,0


In [53]:
for key,val in score.items():
    print(f"{key}: {val}")

faithfulness: 0.9
answer_relevancy: 0.8431298070325102
context_precision: 0.9491269841042763
context_recall: 0.9333333333333333
answer_similarity: 0.9477354700664868
answer_correctness: 0.647171962754717
harmfulness: 0.0
